Understanding the Data

In [ ]:
# Book Figure 4-1

import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

nls97.head()
nls97.tail()
nls97.sample(10,random_state=3).T
nls97[1000:1004].T
nls97.index
nls97.set_index("personid", inplace=True)
nls97.loc[[100061,100139,100284]].T # select based on index
nls97.iloc[[0,1,2]].T # Select based on row number
nls97.iloc[0:3].T # same as above
nls97.iloc[-3:].T #last three
nls97.loc[(nls97['gender']=="Female") & (nls97['maritalstatus']=="Married")]
nls97.size
nls97.shape
nls97.dtypes
nls97.info()
nls97.describe()


Missing Data

In [ ]:
# Book Figure 4-2

import pandas as pd

covidtotals = pd.read_csv("/content/drive/MyDrive/covidtotals.csv")
nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

print(covidtotals.isnull().sum(axis=0))
covidtotals.total_cases_pm.fillna(covidtotals.total_cases/(covidtotals.population/1000000), inplace=True)
covidtotals.total_deaths_pm.fillna(covidtotals.total_deaths/(covidtotals.population/1000000), inplace=True)
covidtotals.isnull().sum(axis=0)

nls97.birthmonth.fillna(int(nls97.birthmonth.mean()), inplace=True)# set a value for the missing data
nls97new= nls97.dropna(thresh=2)# removes rows with less than two non-missing values
nls97.wageincome.fillna(method='ffill', inplace=True)# replace missing values with the nearest non-missing value preceding


Inconsistency of Data

In [ ]:
# Book Figure 4-3

import numpy as np
import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

nls97['birthyear'] = pd.to_numeric(nls97['birthyear'])
nls97['gender'].value_counts()
nls97.filter(like="colenr").apply(lambda x:x.str[0:1]=='3').any(axis=1) # Use lambda to test several columns in one statement
np.where(nls97.govprovidejobs.isnull(),np.nan,np.where(nls97.govprovidejobs.str.contains("not"),"No","Yes")) # pattern exists in a string.
nls97.maritalstatus.str.startswith(' ').any()#leading space
nls97.maritalstatus.str.endswith(' ').any()#tailing space
nls97.maritalstatus = nls97.maritalstatus.str.lower()# Convert text to lowercase
nls97.maritalstatus = nls97.maritalstatus.str.strip()# Remove leading/trailing whitespaces
pd.concat([nls97.weeklyhrstv.head(), nls97.weeklyhrstv.str.findall("\d+").head()],axis=1)# extract numeric values from a text string


Conceptual Inconsistency

In [ ]:
# Book Figure 4-4

import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

#with post-graduate enrollment but no bachelor's enrollment
nobach = nls97.loc[nls97.filter(like="colenr").apply(lambda x: x.str[0:1]=='4').any(axis=1) & ~nls97.filter(like="colenr").apply(lambda x: x.str[0:1]=='3').any(axis=1), "colenrfeb97":"colenroct17"]
print(nobach.head())
#wage income but no weeks worked
nls97.loc[(nls97.weeksworked16==0) & nls97.wageincome>0, ['weeksworked16','wageincome']]
# a high wage income.
highwages = nls97.loc[nls97.wageincome >nls97.wageincome.mean()+(nls97.wageincome.std()*3),['wageincome']]
print(highwages.head())
# large changes in weeks worked for the most recent year
workchanges = nls97.loc[~nls97.loc[:, "weeksworked12":"weeksworked16"].mean(axis=1). between(nls97.weeksworked17*0.5,nls97.weeksworked17*2)  & ~nls97.weeksworked17.isnull(),"weeksworked12":"weeksworked17"]
workchanges.head()

Duplicates

In [ ]:
# Book Figure 4-5

import numpy as np
import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

nls97.set_index("personid", inplace=True)
print('data shape:',nls97.shape)
print('unique indexes:',nls97.index.nunique()) # find unique index in personid
print('duplicated data:',nls97.duplicated().value_counts()) # find duplicates in the database


Outliers

1) Z-score, IQR, Scatter plot

In [ ]:
# Book Figure 4-6

import numpy as np
import pandas as pd
import seaborn as sns

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")
covidtotals = pd.read_csv("/content/drive/MyDrive/covidtotals.csv")


# Z Value approach
z_scores = np.abs((nls97['wageincome'] - nls97['wageincome'].mean()) / nls97['wageincome'].std())
outliers = nls97[z_scores > 3]
print(outliers)

# IQR approach
thirdq,firstq=nls97['wageincome'].quantile(0.75),nls97['wageincome'].quantile(0.25)
IQR=thirdq-firstq
outlierhigh, outlierlow = 1.5*IQR+thirdq,firstq-1.5*IQR
print(nls97['wageincome'].loc[(nls97['wageincome']>outlierhigh)|(nls97['wageincome']<outlierlow)])

# Scatter plot and filter
ax = sns.regplot(x="total_cases", y="total_deaths",data=covidtotals)
covidtotals.loc[(covidtotals.total_cases_pm<7500)  & (covidtotals.total_deaths_pm>250), ['location','total_cases_pm','total_deaths_pm']]

2) Box Plot

In [ ]:
# Book Figure 4-9

import matplotlib.pyplot as plt
import pandas as pd
nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

plt.boxplot(nls97.satverbal.dropna())
plt.ylabel("SAT Verbal")
plt.annotate('outlier threshold',xy=(1.05,780), xytext=(1.15,780), size=7,arrowprops=dict(facecolor='black', headwidth=2,width=0.5, shrink=0.02))
plt.annotate('3rd quartile',xy=(1.08,570), xytext=(1.15,570), size=7,arrowprops=dict(facecolor='black', headwidth=2,width=0.5, shrink=0.02))
plt.annotate('median', xy=(1.08,500),xytext=(1.15,500), size=7,arrowprops=dict(facecolor='black', headwidth=2,width=0.5, shrink=0.02))
plt.annotate('1st quartile',xy=(1.08,430), xytext=(1.15,430), size=7,arrowprops=dict(facecolor='black', headwidth=2,width=0.5, shrink=0.02))
plt.annotate('outlier threshold',xy=(1.05,220), xytext=(1.15,220), size=7,arrowprops=dict(facecolor='black', headwidth=2,width=0.5, shrink=0.02))


3) Violin Plot

In [ ]:
# Book Figure 4-10

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

sns.violinplot(nls97.satverbal, color="wheat",orient="v")
plt.ylabel("SAT Verbal")
plt.text(0.08, 780, "outlier threshold", horizontalalignment='center', size='x-small')
plt.text(0.065, nls97.satverbal.quantile(0.75), "3rd quartile", horizontalalignment='center', size='x-small')
plt.text(0.05, nls97.satverbal.median(), "Median",horizontalalignment='center', size='x-small')
plt.text(0.065, nls97.satverbal.quantile(0.25), "1st quartile", horizontalalignment='center', size='x-small')
plt.text(0.08, 210, "outlier threshold",horizontalalignment='center', size='x-small')
plt.text(-0.4, 500, "frequency",horizontalalignment='center', size='x-small')


4) KNN

In [ ]:
# Book Figure 4-11

import pandas as pd

covidtotals = pd.read_csv("/content/drive/MyDrive/covidtotals.csv")

!pip install pyod
!pip installsklearn
from pyod.models.knn import KNN
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
analysisvars = ['location','total_cases_pm','total_deaths_pm', 'pop_density','median_age','gdp_per_capita']
covidanalysis = covidtotals.loc[:, analysisvars].dropna()
covidanalysisstand = standardizer.fit_transform(covidanalysis.iloc[:, 1:])
clf_name = 'KNN'
clf = KNN(contamination=0.1)
clf.fit(covidanalysisstand)
KNN(algorithm='auto', contamination=0.1, leaf_size=30,method='largest',metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,radius=1.0)
y_pred = clf.labels_
y_scores = clf.decision_scores_
pred = pd.DataFrame(zip(y_pred, y_scores),columns=['outlier','scores'],index=covidanalysis.index)
pred.sample(10, random_state=1)
covidanalysis.join(pred).loc[pred.outlier==1,['location','total_cases_pm','total_deaths_pm','scores']].sort_values(['scores'], ascending=False)


Data Transformation

In [ ]:
# Book Figure 4-13

import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

# replacing values
nls97['gender'].replace(['Female', 'Male'], [1, 2], inplace=True)
print(nls97.head())

# One-hot encode categorical variables
nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")
nls97_encoded = pd.get_dummies(nls97, columns=['gender'])
nls97_encoded.head()


Data Balancing

In [ ]:
# Book Figure 4-15

# Importing libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.style.use('fivethirtyeight')

# data
x = pd.DataFrame({
	# Distribution with lower outliers
	'x1': np.concatenate([np.random.normal(20, 2, 1000), np.random.normal(1, 2, 25)]),
	# Distribution with higher outliers
	'x2': np.concatenate([np.random.normal(30, 2, 1000), np.random.normal(50, 2, 25)]),
})
np.random.normal

scaler = preprocessing.RobustScaler()
robust_df = scaler.fit_transform(x)
robust_df = pd.DataFrame(robust_df, columns =['x1', 'x2'])

scaler = preprocessing.StandardScaler()
standard_df = scaler.fit_transform(x)
standard_df = pd.DataFrame(standard_df, columns =['x1', 'x2'])

scaler = preprocessing.MinMaxScaler()
minmax_df = scaler.fit_transform(x)
minmax_df = pd.DataFrame(minmax_df, columns =['x1', 'x2'])

fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols = 4, figsize =(20, 5))
ax1.set_title('Before Scaling')
ax1.set_xlabel('X1 , X2')

sns.kdeplot(x['x1'], ax = ax1, color ='r')
sns.kdeplot(x['x2'], ax = ax1, color ='b')
ax2.set_title('After Robust Scaling')
ax2.set_xlabel('X1 , X2')

sns.kdeplot(robust_df['x1'], ax = ax2, color ='red')
sns.kdeplot(robust_df['x2'], ax = ax2, color ='blue')
ax3.set_title('After Standard Scaling')
ax3.set_xlabel('X1 , X2')

sns.kdeplot(standard_df['x1'], ax = ax3, color ='black')
sns.kdeplot(standard_df['x2'], ax = ax3, color ='g')
ax4.set_title('After Min-Max Scaling')
ax4.set_xlabel('X1 , X2')

sns.kdeplot(minmax_df['x1'], ax = ax4, color ='black')
sns.kdeplot(minmax_df['x2'], ax = ax4, color ='g')
plt.show()


Data creating and subsitution

In [ ]:
# Book Figure 4-16

import pandas as pd

nls97 = pd.read_csv("/content/drive/MyDrive/nls97.csv")

# create new column/variable
gpaoverall100 = nls97['gpaoverall'] * 100
nls97['childnum'] = nls97.childathome + nls97. childnotathome

# replacing new value in the dataframe
nls97.childnum.value_counts().sort_index()
nls97.iloc[0, 13] = 2
nls97.set_index("personid", inplace=True)
nls97.loc[[100139,100284,100292],'gpaoverall'] = 0
nls97.loc[nls97.gpaoverall>4, 'gpaoverall'] = 4
nls97.weeklyhrscomputer.replace('None',0, inplace=True)
nls97.weeklyhrscomputer.value_counts()

#Delete column and rows
nls97.drop(columns="childnum")
nls97.drop(labels=[100284,100292], axis=0)